In [1]:
import numpy
import pandas

import gensim
from gensim import corpora

from pythainlp.tokenize import word_tokenize
import pythainlp

import re

### Load Corpus

In [2]:
file_directory = '../data/facebook/1/ผู้บริโภค - AllCafe.txt'
corpus = []
labels = []
    
f = open(file_directory, 'r')
data = f.read().splitlines()
f.close()
    
start = False
num = -2

for line in data:
    if line.startswith('comment'):
        num += 1
        if num < 0:
            continue

        comment = ''.join(x for x in line.split(':')[2:])
        corpus.insert(num, comment)
        labels.insert(num, line.split(':')[1])
        start = True

    elif start:
        corpus[num] += line

In [3]:
len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

Total documents 1091
5 clusters


### Preprocess Corpus

#### Tokenize Corpus

In [4]:
def clean(doc):
    while True:
        new_doc = re.sub('[^\u0E00-\u0E7F]+', '', doc)
        if doc == new_doc:
            break
        else:
            doc = new_doc
    return doc

tokenized_corpus = []
for doc in corpus:
    cleaned_doc = clean(doc)
    tokenized_corpus.append(word_tokenize(cleaned_doc, engine='deepcut'))

Using TensorFlow backend.


#### Remove Words

In [5]:
dictionary = corpora.Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

appear_rate = 1
dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        letter_words.append(word)
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 2655 words
filter frequent words: 952 words
filter letter words: 948 words
filter stop words: 720 words


In [24]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
    print(temp_corpus[-1])
idx_corpus = temp_corpus

['ลด', 'ดีด', 'กาแฟ', 'กด', 'แก้ว', 'บาท', 'ลด', 'ดีด', 'โดน', 'ครึ่ง']
['คน', 'พนักงาน', 'ทำ', 'หน้า', 'เหมือน', 'ส้น', 'ตีน']
['สงสาร', 'พนักงาน', 'เงิน', 'เดือน', 'หน้าที่', 'เงิน', 'เดือน', 'ขาย', 'ขาย', 'เติม', 'เงิน', 'ฝาก', 'เงิน']
['ชง', 'รีบ', 'ทำ', 'งาน', 'มัว', 'คุย', 'ชิบ', 'หาย']
['ฝาก', 'สงสาร', 'พนักงาน', 'เค้า', 'หรอ']
['ผัก', 'ชอบ', 'อ้าว', 'กาแฟ', 'ลด', 'หวาน', 'เบาหวาน']
['คน', 'พนักงาน', 'ชง', 'กาแฟ', 'หน้า', 'ส้น', 'ตีน', 'เด็ก', 'แถว', 'บ้าน', 'ทำ', 'งาน', 'ร้าน', 'คอย', 'ตังค์', 'ปกติ', 'หน้า', 'ร้าน', 'แม่ง', 'ไอ้', 'กาแฟ', 'บังคับ', 'เด็ก', 'ชง', 'ขาย', 'หน้า', 'เหี้ย']
['ถุง', 'ใส่', 'น้ำ', 'แก้ว', 'ใส่', 'ถุง', 'ธรรมดา', 'เอียง', 'เอียง', 'หก', 'หก']
['สั่ง', 'หวาน', 'น้ำตาล', 'ซอง']
['ปรับปรุง', 'เรื่อง', 'คิ้ว', 'พนักงาน', 'ดี', 'คิ้ว']
['รอ', 'สั่ง', 'กาแฟ', 'เหมือน', 'ตัว', 'เหี้ย', 'สัก', 'เซเว่น', 'อ่ะ', 'ยืน', 'เกะกะ']
['กิน', 'กาแฟ', 'กด', 'แก้ว', 'นึง', 'ตอน', 'ทุ่ม', 'นึง', 'ขับ', 'รถ', 'กทม', 'ตี', 'เที่ยง', 'หลับ', 'ง่วง', 'หลับ', 'ใส่', 'กิน']
['

In [25]:
unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(idx_corpus):
    for word in vector:
        array[i, dictionary.token2id[word]] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

,กด,กาแฟ,ครึ่ง,ดีด,บาท,ลด,แก้ว,โดน,คน,ตีน,...,คั่ว,พลาสติก,สแตมป์,มั้ยย,รู้จัก,เกิ้นน,เม้น,เกิ๊น,ถ่าย,คอมเมนท์
0,1,1,1,2,1,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
cluster_num = len(clusters)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=cluster_num).fit(word_vectors)
predicted_labels = kmeans.labels_

In [26]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = idx_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,ลดความดีดของกาแฟลงหน่อยครับ เอ่กาแฟกดแก้ว24บาท...,"[ลด, ดีด, กาแฟ, กด, แก้ว, บาท, ลด, ดีด, โดน, ค...",a,1
1,คนที่บอกว่าพนักงานทำหน้าเหมือนส้นตีนแสดงว่าคุณ...,"[คน, พนักงาน, ทำ, หน้า, เหมือน, ส้น, ตีน]",b,0
2,สงสารพนักงานเซเว่นครับ เงินเดือนไม่มาก หน้าที่...,"[สงสาร, พนักงาน, เงิน, เดือน, หน้าที่, เงิน, เ...",b,0
3,ชงให้มันเร็วๆหน่อยบางทีกูรีบไปทำงาน มัวแต่คุยก...,"[ชง, รีบ, ทำ, งาน, มัว, คุย, ชิบ, หาย]",b,2
4,ไม่ฝากไรมากค่ะ สงสารพนักงาน มึงจะให้เค้าเป็นทุ...,"[ฝาก, สงสาร, พนักงาน, เค้า, หรอ]",b,2


In [10]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(cluster_num):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([ 94,  64,  77, 774,  82]))
	c	e	d	a	b	percent
0  |	12	1	0	62	19	0.6595744680851063
1  |	1	0	0	52	11	0.8125
2  |	6	0	0	8	63	0.8181818181818182
3  |	130	58	39	355	192	0.458656330749354
4  |	8	0	0	21	53	0.6463414634146342


In [62]:
for index, value in result[result['predicted_label'] == 2]['comment'].iteritems():
    print(index, '[', value, ']')

1 [ คนที่บอกว่าพนักงานทำหน้าเหมือนส้นตีนแสดงว่าคุณไม่ใช่กลุ่ม  ของเขาครับ ]
2 [ สงสารพนักงานเซเว่นครับ เงินเดือนไม่มาก หน้าที่เกินเงินเดือนครับ 5555 ขายของ ขายกาแก เติมเงิน ล่าสุดกำลังจะรับฝากเงินให้ธออมสิน 5555 ]
6 [ คนที่บอกพนักงานชงกาแฟหน้าหยั่งส้นตีนน่ะเข้าใจเขาบ้าง เด็กแถวบ้านกูเมื่อก่อนมันทำงานร้านนี่คอยคิดตังค์ตามปกติหน้าอย่างแบ๊ว พอร้านแม่งเอาไอ้กาแฟนี่มาลงแล้วบังคับให้เด็กต้องหัดชงขายเท่านั้นแหละ หน้าเหี้ยในทันที ]
12 [ ข้อดีข้อเสียแยกออกไปตามสาขา พนักงานดีและพนักงานไม่ดีก็มีแต่ละสาขา อีกหลายพันสาขาที่เรายังไม่เคยสัมผัส บางทีพนักงานที่ทำดีก็อาจจะกำลังอ่านคอมเม้นพวกคุณแล้วแอบน้อยใจอยู่ก็ได้ ทั้งที่ทำดีแล้วแต่กลับยังไม่ถูกใจ คนทำดีก็ทำไป คนทำไม่ดีนี่ก็ดังเอ๊าดังเอา อยากให้แยกแยะคะ สาขาไหนระบุไปเลย เชื่อว่าพนักงานหลายคนทำเต็มที่และทำด้วยใจถึงขนาดกลับบ้านไปนอนละเมอก็มี จากใจดวงนึง ]
16 [ มึงควรจะจ้างพนักงานชงประจำอย่างเดียวซัก 1 ตำแหน่ง ไม่ใช่ให้เด็ก 7 แม่งทำทุกอย่างจนหน้าเป็นตูดแล้วก็โดนคนด่า ]
19 [ สงสารพนักงาน เซเว่น ครับ คือไอเรื่องกาแฟ อ่ะ ก็พอรับได้ ในราคา 30 40 บาท  เพราะพน

In [27]:
for index, value in result[result['predicted_label'] == 2]['tokenized_comment'].iteritems():
    print(index, value)

3 ['ชง', 'รีบ', 'ทำ', 'งาน', 'มัว', 'คุย', 'ชิบ', 'หาย']
4 ['ฝาก', 'สงสาร', 'พนักงาน', 'เค้า', 'หรอ']
5 ['ผัก', 'ชอบ', 'อ้าว', 'กาแฟ', 'ลด', 'หวาน', 'เบาหวาน']
8 ['สั่ง', 'หวาน', 'น้ำตาล', 'ซอง']
9 ['ปรับปรุง', 'เรื่อง', 'คิ้ว', 'พนักงาน', 'ดี', 'คิ้ว']
13 ['พนักงาน', 'ขี้', 'เกียจ', 'ใส่', 'ฟอง', 'นม', 'กุ', 'วะ']
14 ['เซเว่น', 'สาขา', 'ไหม', 'แถว', 'บ้าน', 'ซัก', 'สาขา']
17 ['สด', 'ชา', 'ผง', 'สำเร็จ', 'ตู้', 'รสชาติ']
18 ['หลอด', 'สี', 'ขาว', 'หลอด', 'สี', 'ดำ', 'เวลา', 'ซื้อ', 'ตอน', 'เช้า', 'หลอด', 'ดู', 'งาม']
20 ['เวลา', 'คิว', 'ระเบียบ', 'คิว', 'ร้าน', 'ค๊ะ', 'แบ่ง', 'ครึ่ง', 'คิว', 'คิว', 'แถว', 'ตัว', 'พุ่ง', 'ตัว', 'แรง']
30 ['กด', 'ขับ', 'รถ', 'กำแพงเพชร', 'ง่วง', 'กรุงเทพ']
32 ['กาแฟ', 'รสชาติ', 'หน้าตา', 'พนักงาน', 'หน้าตา', 'ดี', 'อร่อย', 'หน้า', 'แย่', 'เมิง', 'ตัว', 'ขม', 'แดก']
40 ['สั่ง', 'โกโก้', 'หวาน', 'พนง', 'ฉีก', 'ซอง', 'น้ำ', 'ร้อน', 'ใส่', 'งี้', 'ซื้อ', 'กด']
44 ['เมนู', 'โปรโมชั่น', 'อร่อย']
47 ['เลิก', 'ผลิต', 'สินค้า', 'เลียน', 'ตัว', 'ยกเลิก', 'สินค้า', 